Kütüphaneler içe aktarılıyor.

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

diabetes data seti içe aktarılıp bağımlı ve bağımsız değişkenler ayrılıyor.

In [51]:
diabetes = pd.read_csv("diabetes.csv")
X = diabetes.iloc[:,:-1].values
y = diabetes.iloc[:,-1].values

In [52]:
X.shape

(768, 8)

In [53]:
y.shape

(768,)

Modelin doğruluğun ölçülmesi için veriler test edilecek veri oranı 0.3 olacak şekilde ayrılıyor.

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 1)

In [55]:
X_train.shape

(537, 8)

In [56]:
X_test.shape

(231, 8)

X in değerlerin de standart sapmaya dahil edilmemesi gereken bir öznitelik var mı diye kontrol ediliyor.

In [57]:
df = pd.DataFrame(X)
df.head()

,0,1,2,3,4,5,6,7
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0


Train ve test verilerinin standart sapmaları alınıyor.

In [58]:
from sklearn.preprocessing import StandardScaler
st = StandardScaler()

X_train = st.fit_transform(X_train)
X_test = st.transform(X_test)

In [59]:
print(X_train)

[[ 3.3536084   0.48047259  0.05216975 ...  0.68611992 -0.94690147
   0.81020454]
 [-1.12101735 -0.76891126 -0.2463932  ...  0.64699564  0.39612987
  -0.69526203]
 [-0.82270897 -0.16023707  0.05216975 ... -0.5788985  -0.7936697
  -1.02981015]
 ...
 [ 2.75699163  0.16011776  1.04737957 ...  1.50772982  0.34505262
   0.72656751]
 [ 0.07221618  1.60171451  0.15169073 ...  1.53381267  0.03258
  -0.611625  ]
 [ 1.5637581  -0.60873384  0.3507327  ...  0.13837998  0.59142526
   1.06111563]]


In [60]:
print(X_test)

[[ 0.96714133  0.48047259  0.25121172 ... -0.76147848  0.53734346
   1.47930079]
 [-0.82270897  0.96100484 -0.44543516 ... -0.74843705 -0.86878331
  -0.94617312]
 [ 0.66883295 -0.38448546 -0.44543516 ... -0.89189275 -0.78766061
  -0.52798796]
 ...
 [-0.82270897 -0.96112415 -0.74399811 ... -0.8658099  -0.70353337
  -0.86253609]
 [ 1.86206648  1.50560806  0.25121172 ...  0.80349276  0.20684358
   0.05747126]
 [ 1.5637581  -0.28837901  0.64929565 ...  0.30791853 -0.62541522
   0.22474532]]


Logistic Regresyon modeli içe aktarılıyor ve bir nesne oluşturulup train verileri ile eğitiliyor.

In [61]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

Katsayı değerleri yazdırılıyor.

In [62]:
print(model.coef_)
print(model.intercept_)

[[ 0.33498141  1.03029784 -0.30681406 -0.02318156 -0.07418398  0.67294487
   0.2001099   0.20383277]]
[-0.90342628]


score ile modelin üreteceği değer ve gerçek değerler hem train verileri için hemde test verileri için karşılaştırılıyor.

In [63]:
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)

print("Train Accuracy: " + format(train_accuracy * 100))
print("Test Accuracy: " + format(test_accuracy * 100))

Train Accuracy: 77.46741154562383
Test Accuracy: 78.35497835497836


Grid Search sınıfı içe aktarılıyor. Denenecek parametre dizileri oluşturuluyor. Grid Search sınıfından bir nesne oluşturuluyor gerekli parametreler veriliyor. Tüm parametreleri deneyen grid_search nesnesi en iyi parametreyi bulması için eğitiliyor. En iyi parametreler best_parameters'e aktarılıyor.

In [64]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'C': [0.2, 0.4, 0.6, 0.8, 1],# Parametreler önem sırasına göre dizilmiştir
    'solver': ["lbfgs", "liblinear", "newton-cg", "sag", "saga"],
    'max_iter': [100, 1000, 10000],
    'fit_intercept': [True, False],
    'intercept_scaling': [1, 2, 3],
    'class_weight': [None, 'balanced'],
    'random_state': [13],  # Bu adımdan sonra derlenme süresi oldukça uzun sürüyor
    #'tol': [1e-4, 1e-3, 1e-2],
    #'dual': [True, False],
    #'l1_ratio': [0.2, 0.4, 0.6, 0.8, 1],
    #'n_jobs': [1, -1]
}

grid_search = GridSearchCV(estimator=model,
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_

print("Best Parameters:", best_parameters)

Best Parameters: {'C': 0.4, 'class_weight': None, 'fit_intercept': True, 'intercept_scaling': 1, 'max_iter': 100, 'random_state': 13, 'solver': 'liblinear'}


Model en iyi parametrelerle tekrardan eğitiliyor ve farklı bir model olarak tutuluyor.

In [65]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)

LogisticRegression(C=0.4, random_state=13, solver='liblinear')

Modelin kaybolmaması için joblib kütüphanesi kullanılıyor. best_model.pkl isminde bir dosya halinde tutulması sağlanıyor.

In [66]:
import joblib

joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

En iyi modelin katsayıları yazzdırılıyor.

In [67]:
print(best_model.coef_)
print(best_model.intercept_)

[[ 0.32348424  0.99787246 -0.29385344 -0.02340237 -0.06475733  0.64899212
   0.19570139  0.20267402]]
[-0.86739149]


En iyi model için score değerleri hesaplanıyor. Modelin bulduğu değerler ile gerçek değerler karşılaştırılıyor.

In [68]:
train_accuracy = best_model.score(X_train, y_train)
test_accuracy = best_model.score(X_test, y_test)

print("Train Accuracy with Best Parameters: " + format(train_accuracy * 100))
print("Test Accuracy with Best Parameters: " + format(test_accuracy * 100))

Train Accuracy with Best Parameters: 77.46741154562383
Test Accuracy with Best Parameters: 78.35497835497836


Model kapatıldıktan sonra tekrar kullanılması için best_model.pkl tekrardan çağırılıyor.

In [72]:
import joblib

loaded_model = joblib.load('best_model.pkl')

Çağırılan modelden test yapılıyor.

In [84]:
y_pred1 = loaded_model.predict(st.transform([[1, 85, 66, 29, 94, 28.1, 0.200, 15]]))
y_pred2 = loaded_model.predict(st.transform([[5, 189, 60, 23, 846, 30.1, 0.345, 59]]))

In [85]:
print(y_pred1)
print(y_pred2)

[0]
[1]
